In [149]:
import numpy as np
import cv2 as cv 
import os

In [150]:

base_folder = os.path.join("train","Task2")
path_video1 = os.path.join(base_folder, "03.mp4")

print("path of video = " + path_video1)
# Open the video
cap = cv.VideoCapture(path_video1)

if cap.isOpened() == False:
    print("Error opening video stream or file")
    
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
print('frame_width = ' + str(frame_width))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
print('frame_height = ' + str(frame_height))

length = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
print("total number of frames = " + str(length))

fps = int(cap.get(cv.CAP_PROP_FPS))
print("frames per second = " + str(fps))

cap.release()

path of video = train\Task2\03.mp4
frame_width = 1920
frame_height = 880
total number of frames = 885
frames per second = 60


In [151]:
def get_video_info(video_path):
    
    print("path of video = " + video_path)
    # Open the video
    cap = cv.VideoCapture(video_path)

    if cap.isOpened() == False:
        print("Error opening video stream or file")
        
    frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    print('frame_width = ' + str(frame_width))
    frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    print('frame_height = ' + str(frame_height))

    length = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    print("total number of frames = " + str(length))

    fps = int(cap.get(cv.CAP_PROP_FPS))
    print("frames per second = " + str(fps))

    cap.release()


In [152]:
#get videos and initial bounding boxes file paths
def get_videos_path_and_initial_bb(videos_path):
    
    files = os.listdir(videos_path)

    vids_paths = []
    bb_paths = []
    
    for file_name in files:
            
        path = os.path.join(videos_path,file_name)
        if path.endswith(".mp4"):
            vids_paths.append(path)
        else:
            if path.endswith(".txt"):
                bb_paths.append(path)
            
    return vids_paths, bb_paths
            
    
vids, ibb_paths = get_videos_path_and_initial_bb(base_folder)
print(vids)
print(ibb_paths)

['train\\Task2\\01.mp4', 'train\\Task2\\02.mp4', 'train\\Task2\\03.mp4', 'train\\Task2\\04.mp4', 'train\\Task2\\05.mp4', 'train\\Task2\\06.mp4', 'train\\Task2\\07.mp4', 'train\\Task2\\08.mp4', 'train\\Task2\\09.mp4', 'train\\Task2\\10.mp4', 'train\\Task2\\11.mp4', 'train\\Task2\\12.mp4', 'train\\Task2\\13.mp4', 'train\\Task2\\14.mp4', 'train\\Task2\\15.mp4']
['train\\Task2\\01.txt', 'train\\Task2\\02.txt', 'train\\Task2\\03.txt', 'train\\Task2\\04.txt', 'train\\Task2\\05.txt', 'train\\Task2\\06.txt', 'train\\Task2\\07.txt', 'train\\Task2\\08.txt', 'train\\Task2\\09.txt', 'train\\Task2\\10.txt', 'train\\Task2\\11.txt', 'train\\Task2\\12.txt', 'train\\Task2\\13.txt', 'train\\Task2\\14.txt', 'train\\Task2\\15.txt']


In [153]:
# Display the video
def display_video(video_path: str, max_number_of_frame_to_run: int = None) -> None:
    """
    This function display the video of the screen frame by frame.
    :param video_path: Path to the video
    :param max_number_of_frame_to_run: Set how many frame to be displayed. If None all frames will be displayed.
    """
    
    current_frame = 0 
    
    # Open the video 
    cap = cv.VideoCapture(video_path)

    while cap.isOpened(): 

        ret, frame = cap.read() # Read the frame 
        if ret is True:
            current_frame = current_frame + 1 
            cv.imshow("Video", frame)

            if max_number_of_frame_to_run is not None and current_frame > max_number_of_frame_to_run:
                break

            if cv.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    # after playing the video, release the video capture
    cap.release()
    # close all the frames
    cv.destroyAllWindows()
    
def show_image(image_, window_name='image', timeout=0):
    """
    Show image.
    :param image_
    :param window_name
    :param timeout
    """
    height, width = image_.shape[:2]

    # fx = 0.3
    # fy = 0.3
    # new_width = int(width * fx)
    # new_height = int(height * fy)
    # image_ = cv.resize(image_,(new_width, new_height))
    
    
    cv.imshow(window_name, np.uint8(image_))
    cv.waitKey(timeout) 
    cv.destroyAllWindows()
    
display_video(vids[0])

In [154]:
get_video_info(vids[0])

path of video = train\Task2\01.mp4
frame_width = 1920
frame_height = 880
total number of frames = 884
frames per second = 60


In [155]:
display_video(vids[0])

In [156]:
#Get the moving objects from current frame, according the background( last 3rd frame)

def get_mask_with_moving_objects(background, frame2, threshold=50):
    
    diff = cv.absdiff(background, frame2)
        
    #Apply median blurr to reduce noise
    diff_red = cv.medianBlur(diff, 5)

    # Convert the difference to grayscale
    gray = cv.cvtColor(diff_red, cv.COLOR_BGR2GRAY)

    # Apply a threshold to obtain a binary image
    _, thresh = cv.threshold(gray, threshold, 255, cv.THRESH_BINARY)
    
    
    # Perform dilation
    kernel = np.ones((30, 30), np.uint8)  # You can adjust the size and shape of the kernel
    dilated_img = cv.dilate(thresh, kernel, iterations=1)
    
    result = cv.bitwise_and(frame2,frame2, mask= dilated_img)
    
    return result

In [157]:
#GET BACKGROUND FROM PREVIOUS FRAME
bbox_frames = []

def trackCar(video_path, tracker, initial_bb):
    
    answer = ""
    cap = cv.VideoCapture(video_path)


    success, frame1 = cap.read()
    if not success:
        print("Failed to read video")
        exit()
    background = frame1.copy()
    
    success, frame2 = cap.read()
    if not success:
        print("Failed to read video")
        exit()
    
    cars = get_mask_with_moving_objects(background, frame2)
    background=frame2.copy()
    
    # tracker.init(frame1, initial_bb)
    
    bbox= initial_bb
    x, y, w, h = [int(coord) for coord in bbox]
    
    #initialize the bbox for first two frames
    for i in range(1,3):
        bbox_frames.append([i,x,y,x+w,y+h])
        answer += f"{i} {x} {y} {x+w} {y+h}\n"

    while True:
        
        success, frame2 = cap.read()
        if not success:
            break

        # Subtract the background from the current frame
        cars = get_mask_with_moving_objects(background,frame2)
        
        # Check if all pixel values are 0(no motion detected)
        if np.all(cars == 0) or np.mean(cars[y:y+h,x:x+w]) < 10:
            # Update the bbox on this frame with the last bbox, if no motion was detected on frame/current bounding box.
            success, bbox = tracker.update(frame2)
            i+=1
            answer += f"{i} {x} {y} {x+w} {y+h}\n"
            
            continue
        else:
        
            success, bbox = tracker.update(cars)
        
        #get coordinates from current bbox
        x, y, w, h = [int(coord) for coord in bbox]
        i+=1
        answer += f"{i} {x} {y} {x+w} {y+h}\n"
        
        #draw the current bounding box to current frame
        # cv.rectangle(cars, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv.imshow('Extracted Background', cars)

        if cv.waitKey(25) == ord('q'):
            break
        
        #read next two frame to get a consistent motion.
        ret, _ = cap.read()
        ret, _ =cap.read()
        
        for _ in range(2):
            i+=1
            answer += f"{i} {x} {y} {x+w} {y+h}\n"
        
        #upgrade the background
        background = frame2.copy()
        

    cap.release()
    cv.destroyAllWindows()
    
    return answer


vid_idx =0

# for vid_idx in range(len(vids)):
bb_file = open(ibb_paths[vid_idx])
lines = bb_file.readlines()
bb_coordinates = np.array(lines[1].strip('\n').split())[1:].astype(np.int32)
bbox = (bb_coordinates[0], bb_coordinates[1], bb_coordinates[2]-bb_coordinates[0], bb_coordinates[3]-bb_coordinates[1])

tracker = cv.legacy.TrackerCSRT_create()

result = track_Car(vids[vid_idx], tracker=tracker, initial_bb=bbox)
print(result)

1 1314 673 1643 880
2 1314 673 1643 880
3 1306 667 1648 882
4 1306 667 1648 882
5 1306 667 1648 882
6 1306 667 1648 882
7 1294 663 1643 883
8 1294 663 1643 883
9 1294 663 1643 883
10 1283 654 1619 865
11 1283 654 1619 865
12 1283 654 1619 865
13 1277 653 1606 860
14 1277 653 1606 860
15 1277 653 1606 860
16 1272 650 1601 857
17 1272 650 1601 857
18 1272 650 1601 857
19 1262 643 1591 850
20 1262 643 1591 850
21 1262 643 1591 850
22 1258 642 1581 845
23 1258 642 1581 845
24 1258 642 1581 845
25 1246 634 1562 833
26 1246 634 1562 833
27 1246 634 1562 833
28 1240 630 1563 833
29 1240 630 1563 833
30 1240 630 1563 833
31 1230 626 1540 821
32 1230 626 1540 821
33 1230 626 1540 821
34 1223 622 1539 821



In [158]:
len(vids)

15

In [159]:

#read the initial bounding box from file, track the car and return the full answer with the bounding box for each frame.
def get_file_and_return_bounding_box(video_path, bb_file, tracker):
    
    bb_file = open(bb_file)
    lines = bb_file.readlines()
    bb_coordinates = np.array(lines[1].strip('\n').split())[1:].astype(np.int32)
    
    
    bbox = (bb_coordinates[0], bb_coordinates[1], bb_coordinates[2]-bb_coordinates[0], bb_coordinates[3]-bb_coordinates[1])

    answer = track_Car(video_path,tracker, bbox)
    
    answer = lines[0] + lines[1] + '\n' + answer
    
    return answer
    


In [160]:

def create_predictions_files_names(videos_path):
    
    files = os.listdir(videos_path)

    files_names = []
    
    for file_name in files:
        
        if file_name.endswith(".mp4"):
            
            pred_file_name = file_name.split(".mp4")[0] + "_predicted.txt"
            # print(prediction)
            # print(file_name)
            
            files_names.append(pred_file_name)
            

            
    return files_names
            
    
predicted_file_names = create_predictions_files_names(base_folder)
print(len(predicted_file_names))

15


In [161]:
results = []
#iterate over each video path, initialize the tracker, track cars, and write the answer in files
for i in range(len(vids)):
    
    tracker = cv.legacy.TrackerCSRT_create()
    
    answer = get_file_and_return_bounding_box(vids[i],ibb_paths[i], tracker)
    fileName = predicted_file_names[i]

    
    with open(fileName, "w") as file:
        file.write(answer)

d:\MASTER\ANUL I\Sem_II\Computer Vision\CV_venv\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
d:\MASTER\ANUL I\Sem_II\Computer Vision\CV_venv\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
